In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
dir = "/content/drive/My Drive/NLP_CA4_Question2_Data"

train = pd.read_excel(dir+"/train.xlsx")
test = pd.read_excel(dir+"/test.xlsx")
valid = pd.read_excel(dir+"/valid.xlsx")
del dir

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
encoded_dict = {'bible':0, 'quran':1, 'mizan':2}
train['category'] = train.category.map(encoded_dict)
test['category'] = test.category.map(encoded_dict)
valid['category'] = valid.category.map(encoded_dict)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train.category)
y_test = to_categorical(test.category)
y_valid = to_categorical(valid.category)

In [ ]:
import transformers

In [ ]:
from transformers import AutoConfig,AutoTokenizer,TFBertModel,AutoModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
x_train = tokenizer(
    text=train.source.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test.source.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_valid = tokenizer(
    text=valid.source.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 128
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=3e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
    )

loss = CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = ({'input_ids':x_valid['input_ids'],'attention_mask':x_valid['attention_mask']}, y_valid),
    epochs=10,
    batch_size=32
)

Epoch 1/10
394/394 [==============================] - 357s 906ms/step - loss: 0.0017 - balanced_accuracy: 0.9998 - val_loss: 0.0662 - val_balanced_accuracy: 0.9833
Epoch 2/10
394/394 [==============================] - 366s 929ms/step - loss: 0.0014 - balanced_accuracy: 0.9999 - val_loss: 0.0689 - val_balanced_accuracy: 0.9848
Epoch 3/10
394/394 [==============================] - 365s 927ms/step - loss: 0.0016 - balanced_accuracy: 0.9997 - val_loss: 0.0684 - val_balanced_accuracy: 0.9837
Epoch 4/10
394/394 [==============================] - 366s 928ms/step - loss: 0.0012 - balanced_accuracy: 0.9998 - val_loss: 0.0699 - val_balanced_accuracy: 0.9833
Epoch 5/10
394/394 [==============================] - 366s 929ms/step - loss: 9.5674e-04 - balanced_accuracy: 1.0000 - val_loss: 0.0712 - val_balanced_accuracy: 0.9841
Epoch 6/10
394/394 [==============================] - 365s 928ms/step - loss: 0.0011 - balanced_accuracy: 0.9998 - val_loss: 0.0728 - val_balanced_accuracy: 0.9830
Epoch 7/10
3

In [ ]:
import numpy as np
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = test.category
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       900
           1       0.98      0.98      0.98       900
           2       0.99      0.99      0.99       900

    accuracy                           0.98      2700
   macro avg       0.98      0.98      0.98      2700
weighted avg       0.98      0.98      0.98      2700



In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predicted, pos_label=3)

NameError: ignored